In [16]:
import sqlite3
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

db_path = '../buildfarmer.db'
conn = sqlite3.connect(db_path)

build_time_query = open('get_ros_buildtimes.sql', 'r').read()
df = pd.read_sql_query(build_time_query, conn)

df['domain'] = df['domain'].str.lstrip('https://').str.lstrip('http://').str.rstrip('/')
df['build_datetime'] = df['build_datetime'].str.split('.').str[0] # Some datetimes have .%f at the end
df['build_datetime'] = pd.to_datetime(df['build_datetime'], format='%Y-%m-%d %H:%M:%S')
display(df.head())
conn.close()

,domain,job_name,build_number,buildtime_min,build_datetime
0,ci.ros2.org,nightly_linux_release,2513,184.494050,2023-01-01 01:57:52
1,ci.ros2.org,nightly_linux_repeated,2885,268.839800,2023-01-01 02:11:10
2,ci.ros2.org,packaging_linux,2886,66.857483,2023-01-01 02:35:19
3,ci.ros2.org,packaging_linux-rhel,1385,61.685100,2023-01-01 02:40:16
4,ci.ros2.org,packaging_windows,2716,99.018267,2023-01-01 05:08:01


In [20]:
# Extra filters for data
df = df[df['build_datetime'] >= '2025-01-01']

filter_jobs = [
    'nightly_linux_debug',
    'nightly_linux_release',
    'nightly_linux_repeated',
    'Rci__nightly-release_ubuntu_noble_amd64',
    'Rci__nightly-debug_ubuntu_noble_amd64',
    'Rci__nightly-connext_ubuntu_noble_amd64',
    'Rci__nightly-cyclonedds_ubuntu_noble_amd64',
    'Rci__nightly-fastrtps_ubuntu_noble_amd64',
    'Rci__nightly-zenoh_ubuntu_noble_amd64',
]

df = df[df['job_name'].isin(filter_jobs)]

In [21]:
domains = df['domain'].unique()

for domain in domains:
    domain_df = df[df['domain'] == domain]

    fig = px.line(
        domain_df,
        x='build_datetime',
        y='buildtime_min',
        color='job_name',
        title=domain,
        markers=True,
        labels={
            'build_datetime': 'Date of build',
            'buildtime_min': 'Buildtime (minutes)',
            'job_name': 'Job name',
        },
        hover_data=['build_number']
    )

    fig.update_traces(visible="legendonly")

    fig.update_layout(
        legend_title_text='Click to filter jobs',
        xaxis_title="Date",
        yaxis_title="Buildtime (Minutes)",
        height=700
    )

    fig.show()

    fig.write_html(f'buildtime_line_trend_{domain}.html')


In [22]:
df['month_year'] = df['build_datetime'].dt.to_period('M').astype(str)
df = df.sort_values('month_year')

domains = df['domain'].unique()
for domain in domains:
    domain_df = df[df['domain'] == domain]

    fig = px.box(
        domain_df, 
        x='month_year', 
        y='buildtime_min', 
        color='job_name',
        title=f"{domain}",
        labels={
            "month_year": "Month",
            "buildtime_min": "Buildtime (Minutes)",
            "job_name": "Job Name"
        },
        hover_data=['build_number'] 
    )

    # Hide all by default (click legend to view)
    fig.update_traces(visible="legendonly")

    fig.update_layout(
        legend_title_text='Click to Select Job',
        xaxis_title="Month",
        yaxis_title="Distribution of Time (Minutes)",
        height=600,
        boxmode='group' # Groups boxes side-by-side for the same month
    )
    
    fig.write_html(f'buildtime_boxplot_trend_{domain}.html')
    fig.show()